# Notebook Híbrido: (Local ➔ Colab) Clasificador de Texto

## INSTRUCCIONES IMPORTANTES
Este notebook debe ejecutarse en dos pases con dos kernels diferentes:

## PASE 1 (Preparación de Datos):

**Kernel:** Conecta este notebook a tu Kernel de Python Local.

**Acción:** Ejecuta todas las celdas de la "PARTE 1" (de 1.1 a 1.5).

**Resultado:** Se conectará a tu fuente de datos local (MariaDB o archivos) y creará datos_procesados.parquet.

## PASE 2 (Entrenamiento en Nube):

**Kernel:** Cambia el kernel de este notebook a tu Runtime de Colab (con GPU T4).

**Acción:** Ejecuta todas las celdas de la "PARTE 2" (de 2.1 a 2.9).

**Resultado:** El modelo se entrenará en la GPU de Colab y se guardarán tus predicciones localmente.

------------------------------------------------------------------------

### PARTE 1: PREPARACIÓN DE DATOS (EJECUTAR EN KERNEL LOCAL)

#### Guardián de Kernel (Local)
#### Esta celda verifica que estés en el kernel correcto.

In [3]:
# --- 1.1 Guardián de Kernel (Local) ---
# Esta celda verifica que el kernel actual sea local y no de Colab.
import sys
import os

print("--- Verificando Kernel para la PARTE 1 ---")

# 'sys.modules' es un diccionario de todos los módulos que han sido importados.
# Si 'google.colab' está en ese diccionario, significa que el script corre en Colab.
if 'google.colab' in sys.modules:
    # Si es así, lanzamos una excepción para detener la ejecución.
    raise Exception(
        "¡KERNEL INCORRECTO! "
        "Esta celda (Parte 1) debe ejecutarse en un KERNEL LOCAL, no en Colab. "
        "Por favor, cambie el kernel a su entorno de Python local."
    )
else:
    # Si no, estamos en el kernel local correcto.
    print("Kernel LOCAL detectado. ¡Correcto!")
    print(f"Usando Python en: {sys.executable}")

--- Verificando Kernel para la PARTE 1 ---
Kernel LOCAL detectado. ¡Correcto!
Usando Python en: c:\Program Files\Python313\python.exe


Instalar Librerías (Local)

In [4]:
# --- 1.2 Instalar Librerías (Local) ---
# (Asegúrate de haber ejecutado esto una vez en tu terminal local)
!pip install pandas pyarrow fastparquet openpyxl SQLAlchemy pymysql pydrive2 -q
print("Asegúrate de haber instalado localmente: pandas, pyarrow, fastparquet, openpyxl, SQLAlchemy, pymysql, pydrive2")

Asegúrate de haber instalado localmente: pandas, pyarrow, fastparquet, openpyxl, SQLAlchemy, pymysql, pydrive2



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Importar Librerías (Local)

In [5]:
# --- 1.3 Importar Librerías (Local) ---
import pandas as pd
import os
from sqlalchemy import create_engine # Para conectarse a bases de datos SQL
from pydrive2.auth import GoogleAuth # Para autenticación con Google
from pydrive2.drive import GoogleDrive # Para interactuar con Google Drive
print("Librerías locales (incluyendo PyDrive) importadas.")

Librerías locales (incluyendo PyDrive) importadas.


Configuración del Pipeline

In [6]:
# --- 1.4 CONFIGURACIÓN DEL USUARIO (El Menú) ---
# En esta celda se define de dónde saldrán los datos y cómo se llaman las columnas.

# --- A. Configuración de Columnas (Requerido) ---
COLUMNA_TEXTO = "OBSERVACION"       # Columna con el texto a analizar
COLUMNA_ETIQUETA = "AA"             # Columna con la categoría o 'label'
ETIQUETA_A_EXCLUIR = "OTROS"        # Categoría que queremos reclasificar

# --- B. Configuración de SALIDA (Requerido) ---
# Nombre del archivo Parquet que se creará localmente y se subirá a Drive.
RUTA_ARCHIVO_PARQUET_SALIDA = "datos_procesados.parquet"

# --- C. CONFIGURACIÓN DE FUENTE ORIGINAL (Elige UNA) ---
# Prioridad 1: Base de Datos. Prioridad 2: Archivos.
# Descomenta la sección que quieras usar.

# Opción 1: Base de Datos MariaDB (o MySQL) [OPCIÓN PRIORITARIA]
#------------------------------------------------
TIPO_FUENTE_ORIGINAL = 'MARIADB'
RUTA_FUENTE_ORIGINAL = None # No se usa un archivo

# Configuración de la conexión (se ejecutará desde tu IP local)
MARIADB_USUARIO = "roserom"
MARIADB_CONTRASENA = "roserom"
MARIADB_HOST = "192.168.152.197" # o 127.0.0.1
MARIADB_PUERTO = "3306"
MARIADB_BDD = "reportes"

# La consulta SQL para traer los datos.
# (Esta consulta ahora usa espacios normales)
CONSULTA_SQL = f"""
    SELECT 
        * FROM aa_table
"""

# Opción 2: Archivo (Excel o CSV) [OPCIÓN SECUNDARIA]
#------------------------------------------------
# Descomenta una de estas líneas si NO usas MariaDB

# TIPO_FUENTE_ORIGINAL = 'EXCEL'
# RUTA_FUENTE_ORIGINAL = "mi_archivo_excel.xlsx" 
# NOMBRE_HOJA_EXCEL = 0 # 0 para la primera hoja, o el nombre 'Hoja1'

# TIPO_FUENTE_ORIGINAL = 'CSV'
# RUTA_FUENTE_ORIGINAL = "SOLICITUD DE INFORMACIÓN ACTOS ADMINISTRATIVOS  corte 20251024.xlsx - ACTOS ADMINISTRATIVOS.csv"
# SEPARADOR_CSV = ',' 

#------------------------------------------------

print(f"Configuración cargada. Fuente: {TIPO_FUENTE_ORIGINAL}")
print(f"Salida será: {RUTA_ARCHIVO_PARQUET_SALIDA}")

Configuración cargada. Fuente: MARIADB
Salida será: datos_procesados.parquet


Cargar Datos y Guardar en Parquet (Local)

In [7]:
# --- 1.5 Cargar y guardar en Parquet ---
# Esta celda se conecta a la fuente de datos definida en 1.4
# y guarda el resultado en un archivo Parquet local.
print(f"Cargando desde la fuente: {TIPO_FUENTE_ORIGINAL}...")

try:
    # CASE 1: Fuente es MariaDB (¡Prioridad!)
    if TIPO_FUENTE_ORIGINAL == 'MARIADB':
        print(f"Conectando a MariaDB en {MARIADB_HOST}...")
        # Se crea la cadena de conexión estándar de SQLAlchemy
        CADENA_CONEXION_DB = (
            f"mysql+pymysql://{MARIADB_USUARIO}:{MARIADB_CONTRASENA}"
            f"@{MARIADB_HOST}:{MARIADB_PUERTO}/{MARIADB_BDD}"
        )
        engine = create_engine(CADENA_CONEXION_DB)
        # 'with' asegura que la conexión se cierre automáticamente
        with engine.connect() as connection:
            df_origen = pd.read_sql(CONSULTA_SQL, connection)
        print(f"Datos cargados desde MariaDB.")

    # CASE 2: Fuente es Excel
    elif TIPO_FUENTE_ORIGINAL == 'EXCEL':
        if not os.path.exists(RUTA_FUENTE_ORIGINAL):
            raise FileNotFoundError(f"Archivo Excel no encontrado: {RUTA_FUENTE_ORIGINAL}")
        df_origen = pd.read_excel(RUTA_FUENTE_ORIGINAL, sheet_name=NOMBRE_HOJA_EXCEL)
        print("Datos cargados desde Excel.")

    # CASE 3: Fuente es CSV
    elif TIPO_FUENTE_ORIGINAL == 'CSV':
        if not os.path.exists(RUTA_FUENTE_ORIGINAL):
            raise FileNotFoundError(f"Archivo CSV no encontrado: {RUTA_FUENTE_ORIGINAL}")
        try:
            df_origen = pd.read_csv(RUTA_FUENTE_ORIGINAL, sep=SEPARADOR_CSV)
        except UnicodeDecodeError:
            print("Error de UTF-8, reintentando con encoding 'latin1'...")
            df_origen = pd.read_csv(RUTA_FUENTE_ORIGINAL, sep=SEPARADOR_CSV, encoding='latin1')
        print("Datos cargados desde CSV.")
    
    # Opción por defecto: Error
    else:
        raise ValueError(
            f"TIPO_FUENTE_ORIGINAL ('{TIPO_FUENTE_ORIGINAL}') no reconocido. "
            f"Valores válidos: 'MARIADB', 'EXCEL', 'CSV'."
        )

    # --- Verificación y guardado en Parquet ---
    print(f"Columnas cargadas: {df_origen.columns.tolist()}")
    
    # Verificación crucial: ¿Existen las columnas que el usuario definió?
    if COLUMNA_TEXTO not in df_origen.columns or COLUMNA_ETIQUETA not in df_origen.columns:
        raise ValueError(f"¡ERROR! El DataFrame cargado NO contiene las columnas especificadas: '{COLUMNA_TEXTO}' y '{COLUMNA_ETIQUETA}'.")
    else:
        print("Columnas requeridas verificadas.")
        # Guarda el DataFrame completo en formato Parquet
        df_origen.to_parquet(RUTA_ARCHIVO_PARQUET_SALIDA, index=False)
        print(f"\nArchivo Parquet guardado localmente en: '{RUTA_ARCHIVO_PARQUET_SALIDA}'")
        print(f"Total de filas guardadas: {len(df_origen)}")

except Exception as e:
    print(f"Error fatal al cargar los datos: {e}")
    raise e

Cargando desde la fuente: MARIADB...
Conectando a MariaDB en 192.168.152.197...
Datos cargados desde MariaDB.
Columnas cargadas: ['PROVINCIA_RECEPCION', 'CANTON_RECEPCION', 'EDIFICIO_RECEPCION', 'FECHA_PS', 'DENUNCIANTE', 'USUARIO_INGRESA', 'AA', 'NUMERO_AA', 'PROVINCIA_US', 'CANTON_US', 'EDIFICIO_US', 'ESPECIALIZACION', 'FISCALIA', 'CEDULA_FISCAL', 'NOMBRE_FISCAL', 'OBSERVACION']
Columnas requeridas verificadas.

Archivo Parquet guardado localmente en: 'datos_procesados.parquet'
Total de filas guardadas: 475207


Subir Parquet a Google Drive

In [21]:
# --- 1.6 Subir archivo Parquet a Google Drive ---
# Esta celda toma el archivo Parquet local y lo sube a Google Drive.

# --- ADVERTENCIA DE CONFIGURACIÓN ---
#
# ¡IMPORTANTE! Antes de ejecutar esta celda, debes:
# 1. Haber seguido los pasos para crear credenciales en Google Cloud.
# 2. Haber descargado el archivo JSON de credenciales.
# 3. Haber renombrado ese archivo a 'client_secrets.json'.
# 4. Haber colocado el archivo 'client_secrets.json' EN LA MISMA CARPETA
#    que este notebook.
#
# Si no lo has hecho, esta celda fallará con un error 'FileNotFoundError'.
#
# --- FIN DE LA ADVERTENCIA ---

print("Iniciando autenticación con Google Drive...")




# Autenticación
gauth = GoogleAuth()
# Ajuste: Especificar la ruta al archivo de secretos.
# Como el archivo está un nivel arriba del notebook, usamos '../'.
# Esto resuelve el error 'FileNotFoundError'.
gauth.settings['client_config_file'] = '../client_secrets.json'

# Esto buscará 'client_secrets.json' en esta carpeta.
# Si no existen credenciales guardadas (settings.yaml),
# abrirá tu navegador para que inicies sesión la primera vez.



print("Autenticación exitosa.")

# --- Opcional: Especificar una carpeta de Google Drive ---
# 1. Ve a Google Drive en tu navegador.
# 2. Entra en la carpeta donde quieres guardar los archivos.
# 3. Copia la última parte de la URL.
#    Ejemplo URL: https://drive.google.com/drive/folders/1xmrejogq4heB9AkqeK1xhIJEOJhtH-M6?usp=drive_link
#    El ID es '1xmrejogq4heB9AkqeK1xhIJEOJhtH-M6'
# 4. Descomenta la siguiente línea y pega tu ID:
ID_CARPETA_DRIVE = "1xmrejogq4heB9AkqeK1xhIJEOJhtH-M6" # Corregido: Se eliminó '?usp=drive_link'

# --- Subida del archivo ---
print(f"Subiendo '{RUTA_ARCHIVO_PARQUET_SALIDA}' a Google Drive...")

# Preparamos los metadatos para la búsqueda/creación
file_metadata = {
    'title': RUTA_ARCHIVO_PARQUET_SALIDA,
    'trashed': False
}

# Si definiste un ID de carpeta, lo añadimos a los metadatos
if 'ID_CARPETA_DRIVE' in locals():
    # 'locals()' es un diccionario de todas las variables locales definidas
    # Esto comprueba si la variable 'ID_CARPETA_DRIVE' existe
    file_metadata['parents'] = [{'id': ID_CARPETA_DRIVE}]
    print(f"Buscando en la carpeta especificada (ID: {ID_CARPETA_DRIVE})...")
else:
    # Si no, busca solo en la raíz "Mi Unidad"
    # 'root' es el alias de Google Drive para la carpeta raíz
    file_metadata['q'] = "'root' in parents"
    print("Buscando en la raíz de 'Mi Unidad'...")


# 1. Busca si el archivo ya existe para reemplazarlo
file_list = drive.ListFile(file_metadata).GetList()

if file_list:
    # Si existe, toma el primero y lo usará para sobrescribir
    file_drive = file_list[0]
    print(f"Archivo existente encontrado. Reemplazando '{file_drive['title']}'...")
else:
    # Si no existe, crea un nuevo objeto de archivo
    # (quitamos 'q' y 'trashed' que solo son para búsqueda)
    create_metadata = {'title': RUTA_ARCHIVO_PARQUET_SALIDA}
    if 'ID_CARPETA_DRIVE' in locals():
        create_metadata['parents'] = [{'id': ID_CARPETA_DRIVE}]
        
    file_drive = drive.CreateFile(create_metadata)
    print(f"Archivo no encontrado. Creando nuevo archivo en Google Drive...")

# 2. Asigna el contenido del archivo local y sube
file_drive.SetContentFile(RUTA_ARCHIVO_PARQUET_SALIDA)
file_drive.Upload()

print("¡ÉXITO! Archivo subido/reemplazado en Google Drive.")
print("\n--- PARTE 1 COMPLETADA ---")
print("Ahora, cambie el Kernel de este notebook a COLAB (con GPU) y ejecute las celdas de la PARTE 2.")

Iniciando autenticación con Google Drive...
Autenticación exitosa.
Subiendo 'datos_procesados.parquet' a Google Drive...
Buscando en la carpeta especificada (ID: 1xmrejogq4heB9AkqeK1xhIJEOJhtH-M6)...


NameError: name 'drive' is not defined

### PARTE 2: ENTRENAMIENTO (EJECUTAR EN KERNEL COLAB+GPU)

### Guardián de Kernel (Colab)

In [ ]:
import sys
import torch

print("--- Verificando Kernel para la PARTE 2 ---")
# Esta vez, verificamos que 'google.colab' SÍ esté en los módulos
if 'google.colab' in sys.modules:
    print("Kernel de Colab detectado. ¡Correcto!")
    
    # Verificación secundaria: ¿Está la GPU activa?
    if not torch.cuda.is_available():
        raise Exception(
            "¡GPU NO DETECTADA! "
            "Estás en Colab, pero la GPU no está activa. "
            "Ve a 'Entorno de ejecución' > 'Cambiar tipo de entorno de ejecución' y selecciona 'T4 GPU'."
        )
    else:
        print(f"GPU de Colab activa: {torch.cuda.get_device_name(0)}")
else:
    # Si no estamos en Colab, detenemos la ejecución
    raise Exception(
        "¡KERNEL INCORRECTO! "
        "Esta celda (Parte 2) debe ejecutarse en un KERNEL DE COLAB (con GPU). "
        "Por favor, cambie el kernel."
    )

--- Verificando Kernel para la PARTE 2 ---
Kernel de Colab detectado. ¡Correcto!
GPU de Colab activa: Tesla T4


Instalar Librerías (en Colab)

In [ ]:
# --- 2.2 Instalar las librerías necesarias en el runtime de Colab ---
# El '!' ejecuta comandos de terminal en la máquina virtual de Colab
# '-q' significa 'quiet' (silencioso), para no llenar la salida con texto
!pip install transformers datasets accelerate torch pandas scikit-learn -q
!pip install pyarrow fastparquet -q

Importar Librerías (Colab)

In [ ]:
# --- 2.3 Importar Librerías (Colab) ---
import pandas as pd
import numpy as np
import torch
import os 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments 
)
from datasets import Dataset

# Definimos el dispositivo de cómputo como 'cuda' (la GPU)
device = torch.device("cuda") 
print("Librerías de Colab importadas.")

Librerías de Colab importadas.


Configuración y Carga de Parquet (Colab)

In [ ]:
# --- 2.4 Configuración y Carga de Datos (Colab) ---

# 1. MONTAR GOOGLE DRIVE (Método robusto para Colab)
#    Esto solicitará autenticación y montará tu Drive en '/content/drive'.
from google.colab import auth
from google.colab import drive

auth.authenticate_user()
drive.mount('/content/drive', force_remount=True)


# 2. RUTA_ARCHIVO_PARQUET (¡IMPORTANTE!)
#    Asegúrate de subir 'datos_procesados.parquet' a esta carpeta en tu Google Drive.
#    Puedes crear la carpeta 'Colab_Data' o cambiar la ruta a donde prefieras.
RUTA_ARCHIVO_PARQUET = "/content/drive/MyDrive/Colab_Data/datos_procesados.parquet"

# 3. NOMBRES DE COLUMNAS (deben coincidir con la Parte 1)
COLUMNA_TEXTO = "OBSERVACION"
COLUMNA_ETIQUETA = "AA"
ETIQUETA_A_EXCLUIR = "OTROS"

# 4. CONFIGURACIÓN DEL MODELO
MODELO_BETO = "dcc.uachile/bert-base-spanish-wwm-cased" # Modelo BERT pre-entrenado para español
MODELO_FINAL_NOMBRE = "modelo_clasificador_final" # Carpeta donde se guardará el modelo
SALIDA_PREDICCIONES = f"predicciones_{ETIQUETA_A_EXCLUIR}.csv" # Nombre del archivo final

# --- Cargar datos desde Google Drive ---
print(f"Buscando el archivo en Google Drive: '{RUTA_ARCHIVO_PARQUET}'...")

if not os.path.exists(RUTA_ARCHIVO_PARQUET):
    raise FileNotFoundError(f"¡ARCHIVO NO ENCONTRADO! Asegúrate de haber subido 'datos_procesados.parquet' a la ruta correcta en tu Google Drive.")
else:
    print("¡Archivo encontrado en Google Drive!")

# Leer el archivo Parquet en un DataFrame de Pandas
df = pd.read_parquet(RUTA_ARCHIVO_PARQUET)
print("¡Datos cargados exitosamente desde Parquet!")
print(f"Total de filas: {len(df)}")

ValueError: mount failed